## Preprocessing Binding Affinity from PDBBind Dataset

In [ ]:
# Ki=IC50/(1+([L]/Kd)
import os
import re
import numpy as np
import pandas as pd


_all_protein_ids = os.listdir('./data/pdbbind/v2018')

with open('./data/pdbbind/v2018/index/INDEX_refined_data.2018') as f:
    refinedid2kdki = {line.split()[0]: float(line.split()[3]) for line in f.readlines() if line[0] != "#"}
unit2scale = {'mM': 1e-3, 'uM': 1e-6, 'nM': 1e-9, 'pM': 1e-9, 'fM': 1e-12}

source2type = {'INDEX_general_PL.2018': 'protein-ligand', 
                  'INDEX_general_PN.2018': 'protein-nucleic acid', 
                  'INDEX_general_NL.2018': 'nucleic acid-ligand',
                  'INDEX_general_PP.2018': 'protein-protein'}

def is_protein_id(id, ids=_all_protein_ids):
    return id in ids

def is_refined_data(id, refined_data = refinedid2kdki):
    return id in refined_data

def process_index_file(index_file):
    id2affinity = []
    
    with open(index_file, 'r') as f:
        lines = f.readlines()
  
    for line in lines:
        if not is_protein_id(line[:4]):
            continue
            
        line = line.split('//')[0]

        id, resolution, release_year, binding_data = line.split()
        try: 
            resolution = float(resolution)
        except: 
            resolution = None
        try:
            release_year = int(release_year)
        except:
            release_year = None
        interaction_type = source2type[index_file.split('/')[-1]]
        kdki = refinedid2kdki.get(id)

        #id2affinity.append(dict(zip(['id', 'resolution', 'release_year', 'binding_data', 'source'], 
        #                            line.split() + [source])))
        id2affinity.append(dict(zip(['id', 'resolution', 'release_year', 'binding_data', 'interaction_type', '-log(kd/ki)'], 
                                    [id, resolution, release_year, binding_data, interaction_type, kdki])))
        
    return id2affinity

def get_binding_type(binding_data):
    # Ki=IC50/(1+([L]/Kd)
    if binding_data[:2].lower() == 'kd': return 'kd'
    elif binding_data[:2].lower() == 'ki': return 'ki'
    else: return 'ic50'
    

def get_neg_log_binding_affinity(binding_data):
    binding_affinity_text = re.split('[\=\>\<\~]', binding_data)[-1]
    num, unit = binding_affinity_text[:-2], binding_affinity_text[-2:]
    
    binding_affinity = float(num) * unit2scale.get(unit, 0)

    return -np.log(binding_affinity) if binding_affinity > 0 else 0

In [ ]:
id2affinity = process_index_file('./data/pdbbind/v2018/index/INDEX_general_PL.2018')
#id2affinity = process_index_file('./data/pdbbind/v2018/index/INDEX_general_PN.2018')
#id2affinity += process_index_file('./data/pdbbind/v2018/index/INDEX_general_PP.2018')
#id2affinity += process_index_file('./data/pdbbind/v2018/index/INDEX_general_NL.2018')
#id2affinity += process_index_file('./data/pdbbind/v2018/index/INDEX_refined_data.2018')

binding_affinity_df = pd.DataFrame(id2affinity)
binding_affinity_df['binding_type'] = binding_affinity_df['binding_data'].apply(get_binding_type)
binding_affinity_df['binding_affinity'] = binding_affinity_df['binding_data'].apply(get_neg_log_binding_affinity)
binding_affinity_df = binding_affinity_df.drop('binding_data', axis=1)
binding_affinity_df.to_csv('./data/binding_affinity.csv')

In [ ]:
binding_affinity_df.describe()

In [ ]:
import pandas as pd

binding_affinity_df = pd.read_csv('./data/binding_affinity.csv')

In [ ]:
binding_affinity_df[binding_affinity_df.binding_type=='ic50']

In [ ]:
binding_affinity_df.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(binding_affinity_df[binding_affinity_df['-log(kd/ki)'].notna()]['binding_affinity'].tolist())

In [ ]:
tmp = binding_affinity_df[binding_affinity_df.binding_type=='ic50']

tmp = tmp[tmp.binding_affinity > 18.88 ]

In [ ]:
plt.hist(binding_affinity_df[binding_affinity_df['binding_type'] == 'ic50']['binding_affinity'].tolist())

In [ ]:
binding_affinity_df[binding_affinity_df['binding_type'] == 'ic50']['binding_affinity'].describe()

## Visualize Protein 

In [ ]:
sample_protein_pdb = './data/pdbbind/v2018/1uu3/1uu3_protein_fixed.pdb'
positive_ligand_pdb = './data/pdbbind/v2018/1uu3/1uu3_ligand.pdb'
negative_ligand_pdb = './data/pdbbind/v2018/4mss/4mss_ligand.pdb'

In [ ]:
# This is an interesting package that turns protein to graph. Study it.

#import proteingraph

#p = proteingraph.ProteinGraph(sample_protein_pdb)

### As a graph

In [ ]:
%matplotlib inline
from rdkit.Chem.Draw import IPythonConsole
import networkx as nx
from rdkit.Chem.rdmolfiles import MolFromPDBFile
import matplotlib.pyplot as plt

def mol_to_nx(mol):
    
    """
    https://github.com/maxhodak/keras-molecules/pull/32/commits/dbbb790e74e406faa70b13e8be8104d9e938eba2
    """
    G = nx.Graph()

    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(),
                   atomic_num=atom.GetAtomicNum(),
                   formal_charge=atom.GetFormalCharge(),
                   chiral_tag=atom.GetChiralTag(),
                   hybridization=atom.GetHybridization(),
                   num_explicit_hs=atom.GetNumExplicitHs(),
                   is_aromatic=atom.GetIsAromatic())
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(),
                   bond.GetEndAtomIdx(),
                   bond_type=bond.GetBondType())
    return G

protein = MolFromPDBFile(sample_protein_pdb)
protein_graph = mol_to_nx(protein)
pos = nx.spring_layout(protein_graph) 
nx.draw(protein_graph, width=1, node_size=2)

### 3D Representation of Protein-Ligand Complex

In [ ]:
%load_ext autoreload
%autoreload 2
%pdb off
# set DISPLAY = True when running tutorial
DISPLAY = False
# set PARALLELIZE to true if you want to use ipyparallel
PARALLELIZE = False
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import nglview
import tempfile
import os
import mdtraj as md
import numpy as np
#import deepchem.utils.visualization
#from deepchem.utils.visualization import combine_mdtraj, visualize_complex, convert_lines_to_mdtraj

# https://deepchem.io/docs/notebooks/protein_ligand_complex_notebook.html


"""
def convert_lines_to_mdtraj(molecule_lines):
    molecule_lines = molecule_lines.strip('[').strip(']').replace("'","").replace("\\n", "").split(", ")
    tempdir = tempfile.mkdtemp()
    molecule_file = os.path.join(tempdir, "molecule.pdb")
    with open(molecule_file, "w") as f:
        for line in molecule_lines:
            f.write("%s\n" % line)
    molecule_mdtraj = md.load(molecule_file)
    return molecule_mdtraj
"""
def visualize_protein(molecule_mdtraj):
    traj = nglview.MDTrajTrajectory(molecule_mdtraj) 
    ngltraj = nglview.NGLWidget( traj )
    
    return ngltraj

def visualize_ligand(ligand_mdtraj):
    traj = nglview.MDTrajTrajectory( ligand_mdtraj ) # load file from RCSB PDB
    ngltraj = nglview.NGLWidget( traj )
    ngltraj.representations = [
        { "type": "ball+stick", "params": {"sele": "all" } } ]
    return ngltraj

def combine_mdtraj(protein, ligand):
    chain = protein.topology.add_chain()
    residue = protein.topology.add_residue("LIG", chain, resSeq=1)
    for atom in ligand.topology.atoms:
        protein.topology.add_atom(atom.name, atom.element, residue)
    protein.xyz = np.hstack([protein.xyz, ligand.xyz])
    protein.topology.create_standard_bonds()

    return protein
    
def visualize_complex(complex_mdtraj):
    ligand_atoms = [a.index for a in complex_mdtraj.topology.atoms if "LIG" in str(a.residue)]
    binding_pocket_atoms = md.compute_neighbors(complex_mdtraj, 0.5, ligand_atoms)[0]
    binding_pocket_residues = list(set([complex_mdtraj.topology.atom(a).residue.resSeq for a in binding_pocket_atoms]))
    binding_pocket_residues = [str(r) for r in binding_pocket_residues]
    binding_pocket_residues = " or ".join(binding_pocket_residues)

    traj = nglview.MDTrajTrajectory( complex_mdtraj ) # load file from RCSB PDB
    ngltraj = nglview.NGLWidget( traj )
    ngltraj.representations = [
    { "type": "cartoon", "params": {
    "sele": "protein", "color": "residueindex"
    } },
    { "type": "licorice", "params": {
    "sele": "(not hydrogen) and (%s)" %  binding_pocket_residues
    } },
    { "type": "ball+stick", "params": {
    "sele": "LIG"
    } }
    ]
    return ngltraj
#first_protein, first_ligand = raw_dataset.iloc[0]["protein_pdb"], raw_dataset.iloc[0]["ligand_pdb"]

#protein_mdtraj = convert_lines_to_mdtraj(first_protein)
#ligand_mdtraj = convert_lines_to_mdtraj(first_ligand)
#complex_mdtraj = combine_mdtraj(protein_mdtraj, ligand_mdtraj)

In [ ]:
protein_traj = md.load(sample_protein_pdb)
pos_lig_traj = md.load(positive_ligand_pdb)
neg_lig_traj = md.load(negative_ligand_pdb)

In [ ]:
visualize_protein(protein_traj)

In [ ]:
visualize_ligand(pos_lig_traj)

In [ ]:
visualize_ligand(neg_lig_traj)

In [ ]:
# combination of protein and positive ligand
visualize_complex(combine_mdtraj(md.load(sample_protein_pdb),md.load(positive_ligand_pdb)))

In [ ]:
# combination of protein and negative ligand
visualize_complex(combine_mdtraj(md.load(sample_protein_pdb), md.load(negative_ligand_pdb)))

In [ ]:
# cat 1uu3_protein.pdb 1uu3_ligand.pdb | grep -v 'HOH' > 1uu3_complex.pdb

sample_docked = './data/1uu3_complex.pdb'

docked_traj = md.load(sample_docked)
visualize_complex(docked_traj)

In [ ]:
from utils import pdb2graph
from utils import get_p2l_distance_matrix
from utils import build_adjacency_matrix

protein, ligand, compl = pdb2graph('1y20')

nodes_p, adj_p = protein
nodes_l, adj_l = ligand
nodes_c, adj_c = compl

print(nodes_p.shape)
print(nodes_l.shape)
print(nodes_c.shape)


In [ ]:
build_adjacency_matrix(adj_l)

In [ ]:

get_p2l_distance_matrix(adj_p, adj_l, adj_c)

In [ ]:
from Bio.PDB import PDBParser
import numpy as np

def get_centroid(pdbid):
    ligand_pdb = "./data/pdbbind/v2018/{}/{}_ligand.pdb".format(pdbid, pdbid)

    p = PDBParser()
    s = p.get_structure(pdbid, ligand_pdb)                    


    model = [m for m in s][0]
    chain = [c for c in model][0]
    residue = [r for r in chain][0]

    mat = np.array([atom.get_vector().get_array() for atom in residue])
    
    return np.mean(mat, axis=0)


In [ ]:
centroid = get_centroid("1uu3")

In [ ]:
def get_distance_from_ligand_centroid(pdbid):
    pocket_pdb = "./data/pdbbind/v2018/{}/{}_pocket.pdb".format(pdbid, pdbid)

    centroid = get_centroid(pdbid)
    p = PDBParser()
    s = p.get_structure(pdbid.upper(), pocket_pdb)
    
    model = [m for m in s][0]
    chain = [c for c in model][0]
    residue = [r for r in chain][0]
    
    vectors = [atom.get_vector().get_array() for atom in residue]
    
    return [np.linalg.norm(vector - centroid) for vector in vectors]

In [ ]:
pdbid = '2y2i'
get_distance_from_ligand_centroid("2y2i")

In [ ]:
distance_affinity = []
for i in binding_affinity_df[binding_affinity_df["-log(kd/ki)"].notna()].index.tolist():
    pdbid = binding_affinity_df.loc[i, 'id']
    try:
        distance_affinity.append((np.mean(get_distance_from_ligand_centroid(pdbid)), 
                                  binding_affinity_df.loc[i, '-log(kd/ki)']))
    except:
        continue
    

In [ ]:
distance = [da[0] for da in distance_affinity]
affinity = [da[1] for da in distance_affinity]

plt.plot(distance, affinity, 'o')

In [ ]:
mol = MolFromPDBFile('./data/1uu3_complex.pdb')

## Debugging MDTraj Sanitary Issues

In [ ]:
import mdtraj as md
import os
import numpy as np

data_dir = './data/pdbbind/v2018'
pdbid = '1yhm'

protein_pdb = os.path.join(data_dir, pdbid, '{}_protein.pdb'.format(pdbid))
ligand_pdb = os.path.join(data_dir, pdbid, '{}_ligand.pdb'.format(pdbid))


protein_traj = md.load(protein_pdb)
ligand_traj = md.load(ligand_pdb)

In [ ]:
from collections import Counter
chain = protein_traj.topology.add_chain()
residue = protein_traj.topology.add_residue("LIG", chain, resSeq=1)
for atom in ligand_traj.topology.atoms:
    protein_traj.topology.add_atom(atom.name, atom.element, residue)

bond_source = [str(b[0]) for b in protein_traj.topology.bonds]
bond_dest = [str(b[1]) for b in protein_traj.topology.bonds]

protein_traj.xyz = np.hstack([protein_traj.xyz, ligand_traj.xyz])
protein_traj.topology.create_standard_bonds()
after = [b for b in protein_traj.topology.bonds]
bond_source_after = [str(b[0]) for b in protein_traj.topology.bonds]
bond_dest_after = [str(b[1]) for b in protein_traj.topology.bonds]


In [ ]:
source_over = {b:c for b, c in Counter(bond_source).items() if c > 4}
dest_over = {b:c for b, c in Counter(bond_dest).items() if c > 4}

source_over_after = {b:c for b, c in Counter(bond_source_after).items() if c > 4}
dest_over_after = {b:c for b, c in Counter(bond_dest_after).items() if c > 4}

In [ ]:
source_over

In [ ]:
compl = pdb.select('not water')

In [ ]:
compl

In [ ]:
from io import StringIO
from rdkit.Chem import AllChem
output = StringIO()
#writePDBStream(output, compl)
writePDB('./data/1s9v_compl_prody.pdb', compl)
#pdb_string = output.getvalue()
#rd_mol = AllChem.MolFromPDBBlock(pdb_string)


In [ ]:
visualize_protein(md.load('./data/1s9v_compl_prody.pdb'))

In [ ]:
traj = md.load('./data/1s9v_compl_prody.pdb')

In [ ]:
visualize_complex(traj)